## Tworzenie aplikacji terminalowych
Jednym z podstawowych aspektów tworzenia aplikacji terminalowych jest parsowanie opcji i argumentów. Istnieje wiele różnych rozwiązań tego problemu i wiele różnych zdań co do tego, która z nich jest najlepsza. Na pewno istotna jest:
- łatwość tworzenia kodu obsługującego opcje i parametry
- zgodność z konwencjami systemów operacyjnych
- łatwe dokumentowanie co która opcja lub parametr oznacza
- możliwość łatwej automatyzacji użycia naszego narzędzia
- przenośność i przewidywalne zachowania w różnych terminalach i na różnych systemach operacyjnych.
W bibliotece standardowej Pythona znajdują się przeznaczone do tego 2 moduły `argparse` i `optparse`. Obie biblioteki są używane, jednak posiadają w sobie wiele *magicznych* zachowań lub wypelniają tylko niektóre z powyższych oczekiwań. Na tych zajęciach nauczymy się używać biblioteki `Click` - jest to paczka, którą najpierw należy zainstalować (my już to zrobiliśmy), a która wydaje się na ten moment najwygodniejsza w użyciu.

### Biblioteka Click
`Click` powstał na potrzeby frameworka `Flask` i wewnętrznie opakowuje `optparse`. W założeniu ma być bardziej przenośny i łatwiejszy w użyciu, w szczególności do testowania. Automatycznie generuje strony pomocy, obsługuje zagnieżdżone opcje i argumenty i stroni od niezrozumiałych i magicznych zachowań (auto-poprawianie parametów etc.)
, które utrudniają potem automatyzację workflowów.

#### Dokumentacja
Dokumentacja jest dostępna [tu](https://click.palletsprojects.com/en/8.0.x/#documentation)

#### Podstawy
`Click` definiuje komendy przy użyciu dekoratorów - najprostsza komenda wygląda tak:

In [ ]:
#!pip install click
import click

@click.command()
def hello():
    click.echo('Hello World!')
if __name__ == '__main__':
    hello()

Dekorator zamienia dekorowaną funkcję na komendę - dodając do niej odpowiednie zachowania:
```
$ python hello.py
Hello World!
```
Jednym z nich jest np.: zdolność do wypisania pomocy:
```
$ python hello.py --help
Usage: hello.py [OPTIONS]

Options:
  --help  Show this message and exit.
```

W powyższym przykładzie widać też użycie funkcji `click.echo` zamiast standardowo `print` - click zawiera wiele obejść na problemy, które utrudniają przenaszalność aplikacji między różnymi środowiskami i terminalami. `click.echo` m. in. ulepsza `print` o obsługę unicode'a na różnych systemach operacyjnych, kompatybilne zarządzanie kolorowanym tesktem etc.

Bardziej zaawansowane użycie `click`a to stworzenie grupy komend - podobnie jak np. w `kubectl` mamy grupy komend związanych z konkretnymi zasobami np.: `kubeclt config get-contexts`. Aby utworzyć taką grupę należy:

```
kubectl --namespace="cokolwiek" config  jd get-contexts
kubectl get pods --namespace="asdas"
```

In [ ]:
@click.group()
def cli():
    pass

@click.command()
def initdb():
    click.echo('Initialized the database')

@click.command()
def dropdb():
    click.echo('Dropped the database')

cli.add_command(initdb)
cli.add_command(dropdb)

W powyższym pzykładzie powstała grupa komend `cli`, zawierająca komendy `initdb` i `dropdb` - wywołuje się ją przez:

```
python script.py cli initdb
```

Aby dodać opcje lub argumenty używa się dekoratorów `click.argument` i `@click.option`:

In [ ]:
@click.command()
@click.option('--count', default=1, help='number of greetings')
@click.argument('name', type=str)
def hello(count, name):
    """
    hello - prints your NAME COUNT times
    """
    for x in range(count):
        click.echo(f"Hello {name}!")

#### Argumenty vs opcje
Argumenty nie są tym samym co opcje - zgodnie ze swoja nazwą opcje zazwyczaj są opcjonalne, a argumenty zazwyczaj wymagane. Oczywiście są od tego wyjątki, ale zazwyczaj używa się argumentów by przekazać ścieżki do plików, URL lub subkomendy, a opcji do wszystkiego innego. W `Click` opcje mają więcej funkcjonalności, m.in:
- możliwość poproszenia o brakującą wartość
- łatwe użycie jako flag (włącz/wyłącz)
- możliwość podawania wartości przez zmienne środowiskowe
- w pełni automatyczna dokumentacja

#### Zagnieżdżanie komend i przekazywanie kontekstu
Komendy tworzone z wykorzystaniem Clicka mogą być zagnieżdżone

#### Testy
`Click` udostępnia narzędzie, którym można wywoływać komendę i zbierać jej wyniki w łatwy sposób - `CliRunner`. Nie jest to narzędzie przeznaczone do produkcyjnego użycia w automatyzacji, ale wystarcza do testowania naszego kodu definiującego komendy. Możemy skorzystać z niego np. tak:
```
def test_cli_should_accept_a_path_to_watch_as_an_argument_and_use_default_location_for_checkpoint_store(
        tmpdir_with_file
):
    tmpdir, test_path, *_ = tmpdir_with_file
    runner = CliRunner()
    with runner.isolated_filesystem(temp_dir=tmpdir):
        result = runner.invoke(cli, [str(tmpdir), "watch"])
        assert result.exit_code == 0
        assert store_contains_expected_content("store.json", test_path)

```

### *Zadanie*
```
git checkout task-7
git checkout -b my-solution-7
```
* [ ] Dodaj entrypoint do paczki
    * [ ] W dokumentacji `setuptools` znajdź wyjaśnienie słowa kluczowego `console_scripts`
    * [ ] Dodaj odpowiedni wpis w `setup.cfg`, wskazujący na funkcję `dirwatcher.watcher_cli:cli`
    * [ ] Podbij wersję paczki
    * [ ] Przebuduj paczkę przy użyciu `pyproject-build`
    * [ ] Dla chętnych - wrzuć paczkę do testowego PyPI
* [ ] Dodaj komendę `get`, akceptującą opcje:
    * [ ] `--new` - pokazującą ścieżki nowododanych plików
    * [ ] `--deleted` - pokazującą ścieżki usuniętych plików
    * [ ] `--content-changed` - pokazującą już istniejące pliki, których zawartość się zmieniła
    * [ ] Dla chętnych - czy jesteś w stanie zaimplementować wygodniejszy/bardziej naturalny interfejs?
* [ ] Przetestuj działanie aplikacji przy użyciu testów e2e, podobnie jak jest to zrobione dla komendy `watch`